In [1]:
R.<x> = PolynomialRing(ZZ)
from sage.rings.polynomial.cyclotomic import cyclotomic_coeffs
S = ZZ['x']

In [2]:
def raw_fekete(d):
    "Fekete polynomial with Delta odd "
    if d>0: 
        D=abs(d)
        v=[kronecker(d, a+1) for a in range(0,D-1)]
        F_D=R(v)
        return F_D*x
    else:
        D=abs(d)
        v=[kronecker(d, a+1) for a in range(0,D-1)]
        F_D=-R(v)
        return F_D*x
        
    

def fekete(p):
    #compute f_p(x) 
    F_D=raw_fekete(3*p)
    phi_1= S(cyclotomic_coeffs(1))
    phi_2= S(cyclotomic_coeffs(2))
    phi_3= S(cyclotomic_coeffs(3))
    phi_6= S(cyclotomic_coeffs(6))
    phi_p= S(cyclotomic_coeffs(p))
    if p % 3 == 2:
        factor = x* (phi_1)**2 * phi_2 * (phi_3)**2 * phi_6 * phi_p
    if p % 3 == 1:    
        factor = x* (phi_1)**2 * phi_2 * phi_3 * phi_p
    
    f,r =F_D.quo_rem(factor) 
    return f
    
    
    
def reduced_fekete(d):
    f=fekete(d)
    u=f.trace_polynomial()
    g_D=u[0]
    return g_D

def fekete_reduction(f, q):
    f=f.change_ring(GF(q))
    return f.factor()        

def reduced_fekete_reduction(g,q):
    g=g.change_ring(GF(q))
    return g.factor()

def two_four_cycle(f,q):
    factor = fekete_reduction(f,q)
    number_of_factor = len(factor)
    #check that f is separable modulo q
    for i in range(number_of_factor):
        if factor[i][1] >1:
            return False
    number_two_cycle =0 
    number_four_cycle =0 
    number_even_cycle =0
    for i in range(number_of_factor):
        if factor[i][0].degree() ==2:
            number_two_cycle +=1
        if factor[i][0].degree() == 4:
            number_four_cycle +=1 
        if factor[i][0].degree() % 2 ==0:
            number_even_cycle +=1
    if number_two_cycle ==1 and number_four_cycle ==1 and number_even_cycle ==2:
        return True
    return False
            
    
def search_two_four_cycle(f,n):
    for q in range(n):
        if is_prime(q):
            if two_four_cycle(f,q):
                return q
    return -1 


def almost_cycle(f,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(f,q)
            if len(factor)==3: 
                factor1=factor[0][0]
                factor2=factor[1][0]
                degree1=factor1.degree()
                degree2=factor2.degree()
                if degree1==1 and degree2==1 and factor[0][1]==1 and factor[1][1]==1 and factor[2][1]==1: 
                    return q
    return  -1         
                
def irreducible(f,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(f,q)
            if len(factor)==1 and factor[0][1]==1:
                    return q
    return  -1         
                
       
    
def length_test_2(v):
    #count the number of even entries in v
    count2=0
    for item in v:
        if item==2:
            count2 +=1
    count_even=0     
    for item in v:        
        if item %2 ==0:
            count_even +=1
    if count2==count_even==1:
        return True
    return False    

def length_test_4(v):
    #count the number of even entries in v
    count4=0
    for item in v:
        if item==4:
            count4 +=1
    count_even=0     
    for item in v:        
        if item %2 ==0:
            count_even +=1
    if count4==count_even==1:
        return True
    return False    

    
def two_cycle(f,n):
    result=[]
    for q in range(n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_2(v):
            return q
    return -1    

def two_cycle_2(f,m,n):
    result=[]
    for q in range(m,n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_2(v):
            return q
    return -1 

def four_cycle(f,n):
    result=[]
    for q in range(n):
        v=[]
        if is_prime(q):
            factor=fekete_reduction(f,q)
            for item in factor:
                v.append(item[0].degree())
        if sum(v)==f.degree() and length_test_4(v):
            return q
    return -1  

def cycle(g,n):
    for q in range(n):
        if is_prime(q): 
            factor=fekete_reduction(g,q)
            if len(factor)==2: 
                factor1=factor[0][0]
                coef=factor1.degree()
                if coef==1 and factor[0][1]==1 and factor[1][1]==1: 
                                   return q
    return  -1   
                    

                    
def search_quadruple(f,n):
    irr=irreducible(f,n)
    print(f"f is irreducible at q= ", irr)
    q_cycle=almost_cycle(f,n)
    print(f"f has an (2m-2) cycle at q=", q_cycle)
    q_tranposition=two_cycle(f,n)
    print(f"f has an 2-cycle at q=", q_tranposition)
    q_four_cycle=four_cycle(f,n)
    print(f"f has an 4-cycle at q=", q_four_cycle)
    

def quadruple(f,n):
    irr=irreducible(f,n)
    q_cycle=almost_cycle(f,n)
    q_tranposition=two_cycle(f,n)
    q_four_cycle=four_cycle(f,n)
    result=(irr, q_cycle, q_tranposition, q_four_cycle)
    return result
    
    
def triple(g,n):
    irr=irreducible(g,n)
    q_cycle=cycle(g,n)
    q_tranposition=two_cycle(g,n)
    result=[irr, q_cycle, q_tranposition]
    return result
    
   
    
    
    


# We test the above codes for d. Here we focus on the case $d =3p$ with $p \equiv 3 \pmod{4}$

### We first consider the case d=7

In [6]:
#the Fekete polynomial F_D
p= 7 
d=3*p 
F=raw_fekete(d)
F.factor()

x * (x + 1) * (x - 1)^2 * (x^2 + x + 1) * (x^6 + x^5 + x^4 + x^3 + x^2 + x + 1) * (x^8 - 2*x^7 + 2*x^6 + 2*x^2 - 2*x + 1)

In [7]:
# Fekete polynomial f_D
fekete(p)

x^8 - 2*x^7 + 2*x^6 + 2*x^2 - 2*x + 1

In [5]:
#reduced Fekete polynomial
reduced_fekete(d)

x^3 - 4*x + 2

### Next, we consider $d=3 \times 11$

In [32]:
#the Fekete polynomial F_D
p= 11
d=3*p 
F=raw_fekete(d)
F.factor()

x * (x + 1) * (x - 1)^2 * (x^2 - x + 1) * (x^2 + x + 1)^2 * (x^10 + x^9 + x^8 + x^7 + x^6 + x^5 + x^4 + x^3 + x^2 + x + 1) * (x^12 - x^10 + 2*x^9 - 2*x^8 + 2*x^6 - 2*x^4 + 2*x^3 - x^2 + 1)

In [33]:
f  = fekete(p)
f

x^12 - x^10 + 2*x^9 - 2*x^8 + 2*x^6 - 2*x^4 + 2*x^3 - x^2 + 1

In [35]:
g = reduced_fekete(p)
g

x^6 - 7*x^4 + 2*x^3 + 11*x^2 - 6*x + 2

## Next we consider $d= 3 \times 19$ 

In [37]:
#the Fekete polynomial F_D
p= 19 
d=3*p 
F=raw_fekete(d)
F.factor()

x * (x + 1) * (x - 1)^2 * (x^2 + x + 1) * (x^18 + x^17 + x^16 + x^15 + x^14 + x^13 + x^12 + x^11 + x^10 + x^9 + x^8 + x^7 + x^6 + x^5 + x^4 + x^3 + x^2 + x + 1) * (x^32 + 2*x^29 - 2*x^28 + 2*x^27 + x^26 + 2*x^23 - 2*x^22 + 2*x^21 - x^20 + 2*x^19 - 2*x^18 + 4*x^17 - 4*x^16 + 4*x^15 - 2*x^14 + 2*x^13 - x^12 + 2*x^11 - 2*x^10 + 2*x^9 + x^6 + 2*x^5 - 2*x^4 + 2*x^3 + 1)

In [38]:
f = fekete(p)
f

x^32 + 2*x^29 - 2*x^28 + 2*x^27 + x^26 + 2*x^23 - 2*x^22 + 2*x^21 - x^20 + 2*x^19 - 2*x^18 + 4*x^17 - 4*x^16 + 4*x^15 - 2*x^14 + 2*x^13 - x^12 + 2*x^11 - 2*x^10 + 2*x^9 + x^6 + 2*x^5 - 2*x^4 + 2*x^3 + 1

In [39]:
g = reduced_fekete(p)
g

x^16 - 16*x^14 + 2*x^13 + 102*x^12 - 24*x^11 - 327*x^10 + 108*x^9 + 542*x^8 - 222*x^7 - 415*x^6 + 198*x^5 + 87*x^4 - 52*x^3 + 17*x^2 - 2*x - 2

## Next we consider $d = 3  \times 23$

In [40]:
#the Fekete polynomial F_D
p= 23
d=3*p 
F=raw_fekete(d)
F.factor()

x * (x + 1) * (x - 1)^2 * (x^2 - x + 1) * (x^2 + x + 1)^2 * (x^22 + x^21 + x^20 + x^19 + x^18 + x^17 + x^16 + x^15 + x^14 + x^13 + x^12 + x^11 + x^10 + x^9 + x^8 + x^7 + x^6 + x^5 + x^4 + x^3 + x^2 + x + 1) * (x^36 - 2*x^35 + x^34 + 2*x^33 - 2*x^32 + 2*x^30 - 4*x^29 + 2*x^28 + 2*x^27 - 2*x^26 + 3*x^24 - 4*x^23 + x^22 + 4*x^21 - 2*x^20 - 2*x^19 + 4*x^18 - 2*x^17 - 2*x^16 + 4*x^15 + x^14 - 4*x^13 + 3*x^12 - 2*x^10 + 2*x^9 + 2*x^8 - 4*x^7 + 2*x^6 - 2*x^4 + 2*x^3 + x^2 - 2*x + 1)

In [41]:
f = fekete(p) 
f

x^36 - 2*x^35 + x^34 + 2*x^33 - 2*x^32 + 2*x^30 - 4*x^29 + 2*x^28 + 2*x^27 - 2*x^26 + 3*x^24 - 4*x^23 + x^22 + 4*x^21 - 2*x^20 - 2*x^19 + 4*x^18 - 2*x^17 - 2*x^16 + 4*x^15 + x^14 - 4*x^13 + 3*x^12 - 2*x^10 + 2*x^9 + 2*x^8 - 4*x^7 + 2*x^6 - 2*x^4 + 2*x^3 + x^2 - 2*x + 1

In [43]:
g = reduced_fekete(p)
g

x^18 - 2*x^17 - 17*x^16 + 36*x^15 + 117*x^14 - 268*x^13 - 412*x^12 + 1060*x^11 + 759*x^10 - 2374*x^9 - 616*x^8 + 2950*x^7 - 9*x^6 - 1826*x^5 + 241*x^4 + 432*x^3 - 50*x^2 - 36*x + 4

## Next we consider $d = 3 \times 31$

In [44]:
#the Fekete polynomial F_D
p= 31
d=3*p 
F=raw_fekete(d)
F.factor()

x * (x + 1) * (x - 1)^2 * (x^2 + x + 1) * (x^30 + x^29 + x^28 + x^27 + x^26 + x^25 + x^24 + x^23 + x^22 + x^21 + x^20 + x^19 + x^18 + x^17 + x^16 + x^15 + x^14 + x^13 + x^12 + x^11 + x^10 + x^9 + x^8 + x^7 + x^6 + x^5 + x^4 + x^3 + x^2 + x + 1) * (x^56 - 2*x^55 + 2*x^54 - 2*x^52 + 2*x^51 + x^50 - 4*x^49 + 4*x^48 - 2*x^46 + 2*x^45 + x^44 - 4*x^43 + 4*x^42 - 2*x^40 + 2*x^39 + 3*x^38 - 6*x^37 + 6*x^36 - 2*x^35 + 5*x^32 - 6*x^31 + 6*x^30 + 6*x^26 - 6*x^25 + 5*x^24 - 2*x^21 + 6*x^20 - 6*x^19 + 3*x^18 + 2*x^17 - 2*x^16 + 4*x^14 - 4*x^13 + x^12 + 2*x^11 - 2*x^10 + 4*x^8 - 4*x^7 + x^6 + 2*x^5 - 2*x^4 + 2*x^2 - 2*x + 1)

In [45]:
f = fekete(p) 
f

x^56 - 2*x^55 + 2*x^54 - 2*x^52 + 2*x^51 + x^50 - 4*x^49 + 4*x^48 - 2*x^46 + 2*x^45 + x^44 - 4*x^43 + 4*x^42 - 2*x^40 + 2*x^39 + 3*x^38 - 6*x^37 + 6*x^36 - 2*x^35 + 5*x^32 - 6*x^31 + 6*x^30 + 6*x^26 - 6*x^25 + 5*x^24 - 2*x^21 + 6*x^20 - 6*x^19 + 3*x^18 + 2*x^17 - 2*x^16 + 4*x^14 - 4*x^13 + x^12 + 2*x^11 - 2*x^10 + 4*x^8 - 4*x^7 + x^6 + 2*x^5 - 2*x^4 + 2*x^2 - 2*x + 1

In [46]:
g =reduced_fekete(p)
g

x^28 - 2*x^27 - 26*x^26 + 54*x^25 + 296*x^24 - 646*x^23 - 1929*x^22 + 4504*x^21 + 7871*x^20 - 20246*x^19 - 20507*x^18 + 61262*x^17 + 32541*x^16 - 126382*x^15 - 24630*x^14 + 175946*x^13 - 8820*x^12 - 159634*x^11 + 37592*x^10 + 87884*x^9 - 33928*x^8 - 25354*x^7 + 13830*x^6 + 2486*x^5 - 2492*x^4 + 202*x^3 + 158*x^2 - 26*x - 2

## The Galois groups of $f_{\Delta}(x)$. 

In [50]:
P=Primes()
p = 7
n = 10**5 
while p < 100: 
    if p % 4 ==3:
        f = fekete(p)
        print(p, quadruple(f,n))
    p = P.next(p)  

7 (3, 47, 23, 433)
11 (59, 71, 23, 131)


19 (131, 1091, 23, 113)


23 (677, 587, 17, 1213)


31 (97, 677, 37, 1103)


43 (1193, 827, 1109, 1361)


...

59 (953, 929, 2411, 137)


67 (11, 1109, 233, 137)


71 (263, 487, 73, 823)


79 (107, 2917, 677, 47)


83 (71, 2531, 821, 4373)


In [52]:
P=Primes()
p = 84
n = 10**5 
while p < 200: 
    if p % 4 ==3:
        f = fekete(p)
        print(p, quadruple(f,n))
    p = P.next(p) 

103 (367, 1283, 1427, 5857)


107 (2879, 10337, 683, 4283)


127 (23, 2477, 2887, 3697)


131 (233, 149, 2039, 1259)


139 (3137, 5303, 4007, 467)


151 (3137, 8887, 3733, 5003)


163 (659, 21179, 1151, 1409)


167 (1097, 673, 1187, 4283)


In [3]:
P=Primes()
p = 168
n = 10**5 
while p < 200: 
    if p % 4 ==3:
        f = fekete(p)
        print(p, quadruple(f,n))
    p = P.next(p) 

179 (4139, 557, 2351, 5189)


191 (7247, 5507, 1607, 193)


199 (263, 2417, 967, 1777)


In [5]:
%%time
P=Primes()
p = 200
n = 10**5 
while p < 300: 
    if p % 4 ==3:
        f = fekete(p)
        print(p, quadruple(f,n))
    p = P.next(p) 

211 (1871, 59, 509, 11597)


223 (1783, 1993, 103, 6203)


227 (1223, 1637, 4229, 2663)


239 (11597, 983, 3203, 37)


251 (16889, 863, 827, 4211)


263 (1913, 827, 643, 137)


271 (397, 3067, 67, 4783)


283 (8543, 1019, 479, 2531)
CPU times: user 30min 54s, sys: 2.35 s, total: 30min 57s
Wall time: 31min 36s


In [6]:
%%time
P=Primes()
p = 284
n = 10**5 
while p < 400: 
    if p % 4 ==3:
        f = fekete(p)
        print(p, quadruple(f,n))
    p = P.next(p) 

307 (4001, 28619, 1217, 107)


311 (1523, 13537, 2143, 947)


331 (4889, 29243, 5351, 5297)


In [4]:
%%time
P=Primes()
p = 332
n = 10**5 
while p < 350: 
    if p % 4 ==3:
        f = fekete(p)
        print(p, quadruple(f,n))
    p = P.next(p) 

347 (19751, 3593, 491, 6899)
CPU times: user 22min 50s, sys: 1.12 s, total: 22min 51s
Wall time: 31min 38s


In [5]:
%%time
P=Primes()
p = 350
n = 10**5 
while p < 370: 
    if p % 4 ==3:
        f = fekete(p)
        print(p, quadruple(f,n))
    p = P.next(p)

359 (593, 12497, 173, 2897)


In [3]:
%%time
P=Primes()
p = 360
n = 10**5 
while p < 370: 
    if p % 4 ==3:
        f = fekete(p)
        print(p, quadruple(f,n))
    p = P.next(p)

367 (3583, 2843, 157, 13513)
CPU times: user 17min 26s, sys: 939 ms, total: 17min 27s
Wall time: 28min 47s


In [7]:
%%time
P=Primes()
p = 374
n = 10**5 
while p < 400: 
    if p % 4 ==3:
        f = fekete(p)
        print(p, quadruple(f,n))
    p = P.next(p)

379 (4889, 1889, 6911, 2027)


In [3]:
%%time
P=Primes()
p = 380
n = 10**5 
while p < 400: 
    if p % 4 ==3:
        f = fekete(p)
        print(p, quadruple(f,n))
    p = P.next(p)

383 (2153, 33317, 22963, 20023)
CPU times: user 1h 15min 50s, sys: 3.4 s, total: 1h 15min 53s
Wall time: 1h 17min 9s


In [5]:
%%time
P=Primes()
p = 400
n = 10**5 
while p < 430: 
    if p % 4 ==3:
        f = fekete(p)
        print(p, quadruple(f,n))
    p = P.next(p)

419 (1163, 19469, 17021, 6569)
CPU times: user 49min 30s, sys: 2.08 s, total: 49min 32s
Wall time: 50min 10s


In [3]:
%%time
P=Primes()
p = 420
n = 10**5 
while p < 435: 
    if p % 4 ==3:
        f = fekete(p)
        print(p, quadruple(f,n))
    p = P.next(p)

## The Galois group of $g_{\Delta}(x)$ 

In [4]:
%%time
P=Primes()
p = 7
n = 10**5 
while p < 200: 
    if p % 4 ==3:
        g = reduced_fekete(p)
        print(p, triple(g,n))
    p = P.next(p)

7 [3, 5, 61]
11 [7, 13, 41]
19 [103, 23, 31]
23 [19, 89, 29]
31 [97, 29, 107]
43 [181, 787, 149]


47 [463, 293, 29]
59 [67, 673, 43]


67 [11, 587, 13]
71 [263, 163, 103]
79 [107, 191, 11]


83 [71, 19, 97]


103 [181, 1231, 3]


...

163 [659, 79, 877]


167 [11, 673, 163]


179 [937, 557, 5]


191 [379, 3637, 193]


199 [263, 2417, 97]
CPU times: user 17.5 s, sys: 35.4 ms, total: 17.6 s
Wall time: 18.3 s


In [5]:
%%time
P=Primes()
p = 200
n = 10**5 
while p < 500: 
    if p % 4 ==3:
        g = reduced_fekete(p)
        print(p, triple(g,n))
    p = P.next(p)

211 [1871, 59, 11]


223 [659, 1993, 157]


227 [1223, 61, 23]


239 [719, 983, 37]


251 [2137, 827, 53]


...

467 [421, 3041, 563]


479 [163, 1439, 373]


487 [6703, 311, 151]


491 [10331, 1051, 503]


499 [4691, 5081, 191]
CPU times: user 10min 30s, sys: 1.47 s, total: 10min 32s
Wall time: 10min 43s


In [6]:
%%time
P=Primes()
p = 500
n = 10**5 
while p < 1000: 
    if p % 4 ==3:
        g = reduced_fekete(p)
        print(p, triple(g,n))
    p = P.next(p)

503 [53, 12433, 29]


523 [67, 9127, 839]


547 [4567, 5023, 19]


563 [1451, 2437, 349]


571 [1031, 4127, 1229]


...

907 [2543, 13, 1049]


911 [2273, 7177, 593]


919 [277, 10181, 977]


947 [3169, 28619, 139]


967 [797, 13931, 2221]


In [5]:
%%time
P=Primes()
p = 968
n = 10**5 
while p < 1000: 
    if p % 4 ==3:
        g = reduced_fekete(p)
        print(p, triple(g,n))
    p = P.next(p)

971 [3779, 40961, 367]


983 [2423, 5903, 2161]


991 [179, 5641, 563]
CPU times: user 26min 27s, sys: 1.39 s, total: 26min 28s
Wall time: 26min 55s


In [4]:
%%time
P=Primes()
p = 992
n = 10**5 
while p < 1100: 
    if p % 4 ==3:
        g = reduced_fekete(p)
        print(p, triple(g,n))
    p = P.next(p)

1019 [8713, 12073, 31]


1031 [2239, 24509, 383]


1039 [2857, 1783, 859]


1051 [223, 643, 2027]


1063 [10459, 7951, 23]


1087 [15289, 1063, 1801]


1091 [29573, 3037, 191]
CPU times: user 1h 13min 17s, sys: 5.97 s, total: 1h 13min 23s
Wall time: 2h 18min 18s


In [7]:
factor(2^51 * factorial(51))

2^98 * 3^23 * 5^12 * 7^8 * 11^4 * 13^3 * 17^3 * 19^2 * 23^2 * 29 * 31 * 37 * 41 * 43 * 47

# 2-cycle in the Galois group of f

In [4]:
%%time
P=Primes()
p = 300
n=10**5
while p < 400: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[307, 1217]


[311, 2143]


[331, 5351]


[347, 491]


[359, 173]


[367, 157]


[379, 6911]


[383, 22963]
CPU times: user 40min 2s, sys: 4.15 s, total: 40min 6s
Wall time: 1h 20min 10s


In [5]:
%%time
P=Primes()
p = 400
n=10**5
while p < 500: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[419, 17021]


[431, 8623]


[439, 1217]


[443, 1013]


[463, 2153]


[467, 617]


In [3]:
%%time
P=Primes()
p = 467
n=10**5
while p < 500: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[479, 6673]


[487, 23]


[491, 167]


In [4]:
%%time
P=Primes()
p = 491
n=10**5
while p < 540: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[499, 8039]


[503, 10837]


[523, 2141]
CPU times: user 42min 4s, sys: 3.06 s, total: 42min 7s
Wall time: 1h 21min 59s


In [5]:
%%time
P=Primes()
p = 540
n=10**5
while p < 600: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[547, 3623]


[563, 3527]


[571, 1307]


In [3]:
%%time
P=Primes()
p = 571
n=10**5
while p < 600: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[587, 5009]


In [3]:
%%time
P=Primes()
p = 587
n=10**5
while p < 600: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[599, 20333]
CPU times: user 1h 11min 12s, sys: 8.53 s, total: 1h 11min 21s
Wall time: 2h 24min 28s


In [3]:
%%time
P=Primes()
p = 600
n=10**5
while p < 620: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[607, 2857]


[619, 1619]


[631, 1423]
CPU times: user 15min 54s, sys: 1.64 s, total: 15min 55s
Wall time: 29min 24s


In [2]:
%%time
P=Primes()
p = 630
while p < 660: 
    p = P.next(p)
    if p % 4 ==3:
        print(p)


631
643
647
659
CPU times: user 0 ns, sys: 501 µs, total: 501 µs
Wall time: 327 µs


In [16]:
%%time
p=643
f=fekete(p)
r=2000
m=32*10**3+15*r
n=m+r
print([p, two_cycle_2(f,m,n)])

    

[643, 63647]
CPU times: user 5min 43s, sys: 168 ms, total: 5min 43s
Wall time: 5min 44s


In [20]:
%%time
p=647
f=fekete(p)
r=2000
m=8*10**3
n=2*m
print([p, two_cycle_2(f,m,n)])

    

[647, 9973]
CPU times: user 6min 31s, sys: 160 ms, total: 6min 31s
Wall time: 6min 32s


In [3]:
p=647
f=fekete(p)
n=10**5
print([p, search_two_four_cycle(f,n)])

[647, 10529]


In [0]:
p=643
f=fekete(p)
n=2*10**5
print([p, two_cycle(f,n)])

    

In [8]:
%%time
P=Primes()
p = 640
n=10**5
while p < 660: 
    print(p)
    if p % 4 ==3:
        f = fekete(p)
        print(f.discriminant().is_square())
        #print([p, two_cycle(f ,n)])
    p= P.next(p)    
    

640
641
643
False
647


False
653
659
False
CPU times: user 6.84 s, sys: 16.9 ms, total: 6.85 s
Wall time: 7.24 s


In [4]:
%%time
P=Primes()
p = 631
n=10**5
while p < 650: 
    p = P.next(p)
    print(p)
    if p % 4 ==3:
        f = fekete(p)
        #print([p, two_cycle(f ,n)])

641
643
647
653
CPU times: user 7.95 ms, sys: 2.87 ms, total: 10.8 ms
Wall time: 10.5 ms


In [0]:
%%time
p=643
f=fekete(p)
r=2000
m=10**3
n=2*m
print([p, search_two_four_cycle_2(f,m,n)])

In [3]:
%%time
P=Primes()
p = 647
n=10**5
while p < 660: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[659, 227]
CPU times: user 29.7 s, sys: 41.9 ms, total: 29.8 s
Wall time: 31.1 s


In [7]:
%%time
P=Primes()
p = 659
n=10**5
while p < 680: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[683, 2081]
CPU times: user 7min 8s, sys: 330 ms, total: 7min 8s
Wall time: 7min 21s


In [8]:
%%time
P=Primes()
p = 680
n=10**5
while p < 690: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[683, 2081]


[691, 3527]
CPU times: user 18min 48s, sys: 1.08 s, total: 18min 49s
Wall time: 19min 32s


In [4]:
%%time
P=Primes()
p = 691
n=10**5
while p < 720: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[719, 7]


In [3]:
%%time
P=Primes()
p = 720
n=10**5
while p < 730: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[727, 8887]
CPU times: user 37min 1s, sys: 2.49 s, total: 37min 4s
Wall time: 37min 14s


In [4]:
%%time
P=Primes()
p = 730
n=10**5
while p < 740: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[739, 5711]


[743, 4357]
CPU times: user 42min 32s, sys: 2.56 s, total: 42min 35s
Wall time: 42min 45s


In [9]:
%%time
P=Primes()
p = 743
n=10**5
while p < 770: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[751, 17]
CPU times: user 3.26 s, sys: 16 ms, total: 3.28 s
Wall time: 3.29 s


In [10]:
%%time
P=Primes()
p = 770
n=10**5
while p < 800: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[787, 677]
CPU times: user 2min 53s, sys: 296 ms, total: 2min 53s
Wall time: 2min 54s


In [11]:
%%time
P=Primes()
p = 787
n=10**5
while p < 820: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[811, 2741]
CPU times: user 13min 38s, sys: 947 ms, total: 13min 39s
Wall time: 13min 43s


In [12]:
%%time
P=Primes()
p = 821
n=10**5
while p < 830: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[823, 5647]


In [3]:
%%time
P=Primes()
p = 823
n=10**5
while p < 830: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[827, 4889]


[839, 797]
CPU times: user 28min 41s, sys: 1.13 s, total: 28min 42s
Wall time: 28min 46s


In [5]:
%%time
P=Primes()
p = 840
n=10**5
while p < 860: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[859, 10499]


In [3]:
%%time
P=Primes()
p = 859
n=10**5
while p < 870: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[863, 17317]
CPU times: user 2h 13min 29s, sys: 10.4 s, total: 2h 13min 39s
Wall time: 2h 17min 22s


In [5]:
%%time
P=Primes()
p = 870
n=10**5
while p < 885: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[883, 1019]


[887, 17]
CPU times: user 5min 46s, sys: 378 ms, total: 5min 47s
Wall time: 5min 53s


In [3]:
%%time
P=Primes()
p = 887
n=10**5
while p < 895: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[907, 5519]
CPU times: user 37min 48s, sys: 2.17 s, total: 37min 51s
Wall time: 38min 3s


In [4]:
%%time
P=Primes()
p = 907
n=10**5
while p < 915: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[911, 3823]


In [4]:
%%time
P=Primes()
p = 911
n=10**5
while p < 915: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[919, 7963]
CPU times: user 55min 52s, sys: 4.17 s, total: 55min 57s
Wall time: 56min 26s


In [9]:
%%time
P=Primes()
p = 919
n=10**5
while p < 945: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[947, 47]
CPU times: user 12.1 s, sys: 39.8 ms, total: 12.1 s
Wall time: 12.7 s


In [10]:
%%time
P=Primes()
p = 947
n=10**5
while p < 955: 
    p = P.next(p)
    if p % 4 ==3:
        f = fekete(p)
        print([p, two_cycle(f ,n)])

[967, 10453]
CPU times: user 1h 43min 51s, sys: 7.3 s, total: 1h 43min 58s
Wall time: 1h 48min 5s


In [3]:
p=971
f=fekete(p)
n=10**5
print([p, two_cycle(f,n)])

[971, 9839]


In [4]:
%time
p=983
f=fekete(p)
n=10**5
print([p, two_cycle(f,n)])

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


[983, 1907]


In [3]:
%time
p=991
f=fekete(p)
n=10**5
print([p, two_cycle(f,n)])

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs


[991, 9127]


In [4]:
%%time
p=1019
f=fekete(p)
n=10**5
print([p, two_cycle(f,n)])

[1019, 4943]
CPU times: user 39min 56s, sys: 3.3 s, total: 39min 59s
Wall time: 40min 13s
